In [1]:
print("start quickstart tutorial")

print("arrange libraries")
import torch
from torch import nn # neural network library
import torch.optim.adam
from torch.utils.data import DataLoader # wrap Dataset with iterable
from torchvision import datasets # save samples and labels, download popular datasets
from torchvision.transforms import ToTensor

start quickstart tutorial
arrange libraries


In [2]:
print("Working with data")
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(f"Shape of x [N, C, H, W]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Working with data
Shape of x [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [3]:
print("making models")
device = (
    "cuda"
    if torch.cuda.is_available() else "mps"
    if torch.backends.mps.is_available() else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

print("optimize parameters of model")
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg loss: {test_loss:>8f}\n")

epochs = 5
for t in range(5):
    print(f"Epoch {t+1}\n------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")

making models
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)
optimize parameters of model
Epoch 1
------------------------------
loss: 2.310480 [   64/60000]
loss: 2.271577 [ 6464/60000]
loss: 2.236566 [12864/60000]
loss: 2.221515 [19264/60000]
loss: 2.178337 [25664/60000]
loss: 2.137704 [32064/60000]
loss: 2.142795 [38464/60000]
loss: 2.073173 [44864/60000]
loss: 2.063102 [51264/60000]
loss: 1.998518 [57664/60000]
Test Error: 
 Accuracy: 54.1, Avg loss: 1.991430

Epoch 2
------------------------------
loss: 2.028053 [   64/60000]
loss: 1.982789 [ 6464/60000]
loss: 1.878382 [12864/60000]
loss: 1.891073 [19264/60000]
loss: 1.780962 [25664/60000]
loss: 1.736648 [32064/60000]
loss: 1.730743 [38464/60000]
loss: 1.630619 [44864/60000]
loss

In [7]:
print("Save model")
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch model state to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Save model
Saved Pytorch model state to model.pth
Predicted: "Ankle boot", Actual: "Ankle boot"
